In [2]:
import get_api_data,git_local
import pandas as pd
from flatten_dict import flatten
from datetime import datetime
from dateutil import relativedelta

In [3]:
access_token = 'ef66b5bf5fd6c14d63586bf00caa9e72c78cc675'
repo_owner = 'rspec'
git_url = 'git://github.com/rspec/rspec-rails.git'
api_base_url = 'https://api.github.com'
repo_name = 'rspec-rails'

In [4]:
local = git_local.git_local(git_url,repo_name)
repo = local.clone_repo()
commits = local.get_commits()
commit_df = pd.DataFrame(commits, columns = ['commit_id','commit_time','author','author_email',
                                             'committer','committer_email','comments','commit_parent'])
committed_files = local.get_committed_files()
committed_files_df = pd.DataFrame(committed_files, columns = ['commit_id','file_id','modification_type','file_name'])

In [7]:
def isBuggyCommit(commit):
    res=re.search(r'\b{bug|fix|issue|error|correct|proper|deprecat|broke|optimize|patch|solve|slow|obsolete|vulnerab|debug|perf|memory|minor|wart|better|complex|break|investigat|compile|defect|inconsist|crash|problem|resol|#}\b',utils().stemming(commit),re.IGNORECASE)
    if res is not None:
        return True

In [9]:
def buggy_commits(commits):
    commit['isBuggy'] = pd.Series([0]*commit.shape[0])
    for i in range(commits.shape[0]):
        result = isBuggyCommit(commits.loc[i,'message'])
        if result:
            commits.loc[i,'isBuggy'] = 1
        else:
            commits.loc[i,'isBuggy'] = 0
    return commits

In [ ]:
def get_diffs(repo,commits):
    diffs = {}
    for i in range(len(commits)):
        t0 = repo.get(commits[i])
        files = {}
        if len(t0.parents) == 0:
            continue
        if i != 0:
            t1 = t0.parents[0]
        else:
            continue
        _diff = repo.diff(t1,t0)
        for diff_i in _diff.__iter__():
            file_path = diff_i.delta.new_file.path
            old_lineno = []
            new_lineno = []
            for x in diff_i.hunks:
                for y in x.lines:
                    old_lineno.append(y.old_lineno)
                    new_lineno.append(y.new_lineno)
            files[diff_i.delta.new_file.id] = {'file_path':file_path, 'old_lines':old_lineno,'new_lines':new_lineno}
        diffs[t0.id] = {'files':files,'object':t0}
    return diffs